In [11]:
import pandas as pd
import geopandas as gpd
import rasterio
import glob
import os
import numpy as np
from shapely.geometry import box
from rasterio.mask import mask

In [12]:
#data_dir = "D:/Projects/ianis/change_detection/data"
tif_paths = glob.glob(f"C:/Users/apsam/Documents/work/Thiyaku work/blog_urban_growth/New folder/*.tif")
gpkg_dir = f"C:/Users/apsam/Documents/work/Thiyaku work/blog_urban_growth/gpkg"
# tif_paths_2022 = glob.glob(f"{tif_path}/2022????.tif")
# tif_paths = glob.glob(f"{tif_path}/corrected_rgb_image_inpainted_40.tif")
csv_file = f"C:/Users/apsam/Documents/work/Thiyaku work/blog_urban_growth/cbe_extracted.csv"

In [13]:
def extract_date_from_filename(filename):
    """
    Extracts the date from the filename.

    Parameters:
    filename (str): The name of the file.

    Returns:
    str: The extracted date in YYYY-MM-DD format.
    """
    date_str = filename[:8]
    date_formatted = f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:]}"
    return date_formatted

In [14]:
def extract_all_pixel_values_for_polygons(raster_path, polygons, land_use_info, ras_date):
    """
    Extracts all pixel values within specific polygons from a raster file.

    Parameters:
    raster_path (str): Path to the raster file.
    polygons (list of Polygon): List of shapely Polygon geometries.
    land_use_info (list of str): List of land use names corresponding to each polygon.
    ras_date (str): Date associated with the raster data.

    Returns:
    pd.DataFrame: DataFrame containing all pixel values within each polygon for the specified bands,
                  along with the date, land use information, and pixel coordinates.
    """
    
    values_list = []
    band_names = ['B1', 'B2', 'B3','B4','B5','B6','B7','B8','B9','B10','B11','B12','B13']
    
    with rasterio.open(raster_path) as src:
        for polygon, land_use in zip(polygons, land_use_info):
            # Mask the raster with the polygon
            masked_raster, masked_transform = mask(src, [polygon], nodata=-99, crop=True)
            # Loop through each band to extract pixel values
            for band_index, band_name in enumerate(band_names, start=1):
                band_data = masked_raster[band_index - 1]  # Get the data for the specific band
                
                # Get the coordinates of each pixel in the masked region
                rows, cols = np.where(~np.isnan(band_data))  # Only get pixels within the polygon (non-NaN)
                for row, col in zip(rows, cols):
                    # Get the raster value at this pixel
                    pixel_value = band_data[row, col]
                    # Convert the row/col to actual x, y coordinates
                    x, y = rasterio.transform.xy(masked_transform, row, col, offset='center')
                    # Store the values and metadata
                    values_dict = {
                        'Date': ras_date,
                        'Land Use': land_use,
                        'Band': band_name,
                        "x": x,
                        "y": y,
                        'Value': pixel_value,
                    }
                    values_list.append(values_dict)
    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(values_list)
    
    return df

In [15]:
df_list = []
for tif_path in tif_paths:
    tif_bn = os.path.basename(tif_path)
    tif_date = tif_bn.replace(".tif", "")
    gpkg_bn = f"{tif_date}_training.gpkg"
    gpkg_path = f"{gpkg_dir}/{gpkg_bn}"
    
    training_data = gpd.read_file(gpkg_path)
    # Read the raster file to get its CRS
    with rasterio.open(tif_path) as src:
        raster_crs = src.crs
    if training_data.crs != raster_crs:
        training_data = training_data.to_crs(raster_crs)
    training_data = training_data[~training_data.geometry.isna()]
    df = extract_all_pixel_values_for_polygons(tif_path, training_data.geometry, training_data["label"], tif_date)
    print(tif_bn)
    df_list.append(df)

20230222T050821_20230222T052224_T43PGN.tif
20230227T050749_20230227T052458_T43PGN.tif
20240207T051001_20240207T052026_T43PGN.tif


In [16]:
df.head()

,Date,Land Use,Band,x,y,Value
0,20240207T051001_20240207T052026_T43PGN,settelements,B1,711905.0,1220305.0,-99.0
1,20240207T051001_20240207T052026_T43PGN,settelements,B1,711915.0,1220305.0,-99.0
2,20240207T051001_20240207T052026_T43PGN,settelements,B1,711925.0,1220305.0,-99.0
3,20240207T051001_20240207T052026_T43PGN,settelements,B1,711935.0,1220305.0,-99.0
4,20240207T051001_20240207T052026_T43PGN,settelements,B1,711945.0,1220305.0,-99.0


In [17]:
merged_df = gpd.GeoDataFrame(pd.concat(df_list, ignore_index=True))
merged_df.shape

(3369002, 6)

In [18]:
#csv_file = f"C:/Users/apsam/Documents/work/New folder/wv_data_extracted.csv"
merged_df.to_csv(csv_file, index=False)

In [19]:
merged_df.head()

,Date,Land Use,Band,x,y,Value
0,20230222T050821_20230222T052224_T43PGN,settelements,B1,713455.0,1216585.0,-99.0
1,20230222T050821_20230222T052224_T43PGN,settelements,B1,713465.0,1216585.0,-99.0
2,20230222T050821_20230222T052224_T43PGN,settelements,B1,713475.0,1216585.0,-99.0
3,20230222T050821_20230222T052224_T43PGN,settelements,B1,713485.0,1216585.0,-99.0
4,20230222T050821_20230222T052224_T43PGN,settelements,B1,713495.0,1216585.0,-99.0


In [20]:
merged_df.shape

(3369002, 6)